# Observing intersecting gene variants across patient sets

We visualize overlapping variants using [UpSetR](https://github.com/hms-dbmi/UpSetR) plots ([Conway et al. 2017](https://doi.org/10.1093/bioinformatics/btx364)).

In [1]:
library(UpSetR)
library(reshape2)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
cosmic_file <- file.path('results', 'all_cosmic_variants.tsv')
cosmic_df <- readr::read_tsv(cosmic_file) %>%
    dplyr::mutate(base_sample_id = sapply(final_id, function(x) unlist(strsplit(x, '-'))[1]))
head(cosmic_df)

Parsed with column specification:
cols(
  .default = col_character(),
  Start = col_integer(),
  End = col_integer(),
  depth = col_integer()
)
See spec(...) for full column specifications.


Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,⋯,phastCons7way_vertebrate,phastCons20way_mammalian,SiPhy_29way_logOdds,Otherinfo,sample_name,final_id,het,quality,depth,base_sample_id
1,874779,874826,CCTCCCCAGCCACGGTGAGGACCCACCCTGGCATGATCCCCCTCATCA,-,exonic,SAMD11,.,nonframeshift deletion,SAMD11:NM_152486:exon7:c.645_692del:p.G220Dfs*447,⋯,.,.,.,het . 20,019-F0,008-M2-F0,het,.,20,008
1,26510311,26510311,C,-,exonic,CNKSR1,.,frameshift deletion,"CNKSR1:NM_001297647:exon9:c.866delC:p.P291Hfs*74,CNKSR1:NM_001297648:exon9:c.71delC:p.P26Hfs*74,CNKSR1:NM_006314:exon9:c.845delC:p.P284Hfs*74",⋯,.,.,.,het . 31,019-F0,008-M2-F0,het,.,31,008
1,158576883,158576883,G,A,exonic,OR10Z1,.,nonsynonymous SNV,OR10Z1:NM_001004478:exon1:c.G655A:p.A219T,⋯,0.822,0.761,4.671,het . 127,019-F0,008-M2-F0,het,.,127,008
1,159032487,159032487,T,-,exonic,AIM2,.,frameshift deletion,"AIM2:NM_001348247:exon5:c.712delA:p.T238HREVKRTNSSQLV,AIM2:NM_004833:exon6:c.1027delA:p.T343HREVKRTNSSQLV",⋯,.,.,.,het . 87,019-F0,008-M2-F0,het,.,87,008
1,183187613,183187613,C,T,exonic,LAMC2,.,nonsynonymous SNV,"LAMC2:NM_005562:exon4:c.C493T:p.R165C,LAMC2:NM_018891:exon4:c.C493T:p.R165C",⋯,0.998,0.995,16.012,het . 83,019-F0,008-M2-F0,het,.,83,008
1,208084350,208084350,G,A,exonic,CD34,.,synonymous SNV,"CD34:NM_001025109:exon1:c.C76T:p.L26L,CD34:NM_001773:exon1:c.C76T:p.L26L",⋯,.,.,.,het . 59,019-F0,008-M2-F0,het,.,59,008


In [3]:
for (sample_group in unique(cosmic_df$base_sample_id)) {
    
    upset_fig_file <- file.path('figures', 'upset', paste0('upset_sample_', sample_group, '.pdf'))
   
    
    patient_df <- cosmic_df %>% dplyr::filter(base_sample_id == sample_group)
    
    sample_set <- sort(unique(patient_df$final_id), decreasing = TRUE)

    patient_df_melt <- reshape2::melt(patient_df, id.vars = 'final_id', measure.vars = 'Gene.refGene')
    patient_pivot <- reshape2::dcast(patient_df_melt, value ~ final_id, fun.aggregate = function(x) length(x) )
    patient_pivot[patient_pivot == 2] <- 1
    
    pdf(upset_fig_file, height = 6, width = 7, onefile = FALSE)         
    upset(patient_pivot, order.by = 'freq', sets = sample_set, keep.order = TRUE,
          queries = list(list(query = intersects, params = sample_set,
                              color = 'orange', active = T)), mb.ratio = c(0.7, 0.3),
          text.scale = c(1.8, 1.5, 1.8, 1.5, 1.8, 1.2))
    dev.off()
}

## Generate UpSetR plots before COSMIC filtering


In [4]:
file = file.path('results', 'all_cosmic_prefiltered_variants.tsv')
prefiltered_cosmic_df <- readr::read_tsv(file) %>%
    dplyr::mutate(base_sample_id = sapply(final_id, function(x) unlist(strsplit(x, '-'))[1]))
head(prefiltered_cosmic_df)

Parsed with column specification:
cols(
  .default = col_character(),
  Chr = col_integer(),
  Start = col_integer(),
  End = col_integer(),
  gnomAD_exome_ALL = col_double(),
  gnomAD_exome_AFR = col_double(),
  gnomAD_exome_AMR = col_double(),
  gnomAD_exome_ASJ = col_double(),
  gnomAD_exome_EAS = col_double(),
  gnomAD_exome_FIN = col_double(),
  gnomAD_exome_NFE = col_double(),
  gnomAD_exome_OTH = col_double(),
  gnomAD_exome_SAS = col_double(),
  depth = col_integer()
)
See spec(...) for full column specifications.
Warning message in rbind(names(probs), probs_f):
“number of columns of result is not a multiple of vector length (arg 1)”Warning message:
“5549680 parsing failures.
row # A tibble: 5 x 5 col     row col   expected   actual file                                          expected   <int> <chr> <chr>      <chr>  <chr>                                         actual 1  2061 Chr   an integer X      'results/all_cosmic_prefiltered_variants.tsv' file 2  2062 Chr   an integer X

Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,⋯,phastCons7way_vertebrate,phastCons20way_mammalian,SiPhy_29way_logOdds,Otherinfo,sample_name,final_id,het,quality,depth,base_sample_id
1,17379,17379,G,A,ncRNA_exonic,MIR6859-1;MIR6859-2;MIR6859-3;MIR6859-4,.,.,.,⋯,.,.,.,het . 34,019-F0,008-M2-F0,het,.,34,008
1,139058,139058,C,G,ncRNA_exonic,LOC729737,.,.,.,⋯,.,.,.,het . 125,019-F0,008-M2-F0,het,.,125,008
1,874779,874826,CCTCCCCAGCCACGGTGAGGACCCACCCTGGCATGATCCCCCTCATCA,-,exonic,SAMD11,.,nonframeshift deletion,SAMD11:NM_152486:exon7:c.645_692del:p.G220Dfs*447,⋯,.,.,.,het . 20,019-F0,008-M2-F0,het,.,20,008
1,896101,896101,G,A,exonic,KLHL17,.,nonsynonymous SNV,KLHL17:NM_198317:exon1:c.G28A:p.G10S,⋯,0.009,0.998,7.535,het . 22,019-F0,008-M2-F0,het,.,22,008
1,979690,979690,G,A,intronic,AGRN,.,.,.,⋯,.,.,.,het . 83,019-F0,008-M2-F0,het,.,83,008
1,979835,979835,G,A,intronic,AGRN,.,.,.,⋯,.,.,.,het . 24,019-F0,008-M2-F0,het,.,24,008


In [5]:
for (sample_group in unique(prefiltered_cosmic_df$base_sample_id)) {
    
    upset_fig_file <- file.path('figures', 'upset', 'prefiltered',
                                paste0('upset_sample_', sample_group, '.pdf'))
   
    
    patient_df <- prefiltered_cosmic_df %>% dplyr::filter(base_sample_id == sample_group)
    
    sample_set <- sort(unique(patient_df$final_id), decreasing = TRUE)

    patient_df_melt <- reshape2::melt(patient_df, id.vars = 'final_id', measure.vars = 'Gene.refGene')
    patient_pivot <- reshape2::dcast(patient_df_melt, value ~ final_id, fun.aggregate = function(x) length(x) )
    patient_pivot[patient_pivot >= 2] <- 1
    
    pdf(upset_fig_file, height = 6, width = 7, onefile = FALSE)         
    upset(patient_pivot, order.by = 'freq', sets = sample_set, keep.order = TRUE,
          queries = list(list(query = intersects, params = sample_set,
                              color = 'orange', active = T)), mb.ratio = c(0.7, 0.3),
          text.scale = c(1.8, 1.5, 1.8, 1.5, 1.8, 1.2))
    dev.off()
}